# Laboratorio 2 - Clasificador de rostros

**Autores**

*   [número_estudiante1] Nombre1, Apellido1
*   [número_estudiante2] Nombre2, Apellido2
*   [número_estudiante3] Nombre3, Apellido3

**Formato de entrega**:

* Esta misma notebook en formato .ipynb
* Cambiar el nombre de la notebook por NumEst1_NumEst2_NumEst3_Lab_1.
* Es importante que la notebook pueda ejecutarse sin problemas al seleccionar 'Ejecutar todo'.
* Se considerará que sus datos pueden estar en otra localización.


**Plazo de entrega**: hasta el Domingo 16/06 a las 23:59 horas a través de Aulas.

**Objetivo**: implementar un algoritmo de clasificación que permita predecir si una imagen dada es un rostro o no.

## Librerías

In [34]:
import os
from tqdm import tqdm
from time import time

import random
import numpy as np
import matplotlib.pyplot as plt

from skimage.exposure import equalize_hist

from skimage.transform import integral_image
from skimage.feature import haar_like_feature, haar_like_feature_coord

from sklearn.feature_selection import SelectPercentile, f_classif

## Funciones auxiliares

In [20]:
def extract_feature_image(img, feature_type=None, feature_coord=None):
    """Extrae las Haar features de la imagen"""
    ii = integral_image(img)
    return haar_like_feature(ii, 0, 0, ii.shape[0], ii.shape[1],
                             feature_type=feature_type,
                             feature_coord=feature_coord)

## Datos

**CBCL FACE DATABASE #1**:

*   19 x 19 Grayscale PGM format images
*   Training set:  2429 faces, 4548 non-faces
*   Test set: 472 faces, 23573 non-faces



In [ ]:
!unzip /content/CBCL.zip

In [3]:
suffix = '.pgm'

train_faces = os.listdir('/content/train/face')
train_faces = [filename for filename in train_faces if filename.endswith(suffix)]

train_background = os.listdir('/content/train/non-face')
train_background = [filename for filename in train_background if filename.endswith(suffix)]

test_faces = os.listdir('/content/test/face')
test_faces = [filename for filename in test_faces if filename.endswith(suffix)]

test_background = os.listdir('/content/test/non-face')
test_background = [filename for filename in test_background if filename.endswith(suffix)]

In [4]:
print(f'# Train Faces: {len(train_faces)}')
print(f'# Train Back: {len(train_background)}')
print(f'# Test Faces: {len(test_faces)}')
print(f'# Test Back: {len(test_background)}')

# Train Faces: 2429
# Train Back: 4548
# Test Faces: 472
# Test Back: 23573


In [5]:
# Tomaremos una fracción de los datos. Puede ajustar estos parámetros a gusto
f = 0.2
n_face = int(f*len(train_faces))
n_back = int(f*len(train_background))

# Para mantener la proporción de background en test calculamos:
m = int(np.round(len(test_faces)*len(train_background)/len(train_faces)))

print(f'# Train Faces Sample Size: {n_face}')
print(f'# Train Back Sample Size: {n_back}')
print(f'# m: {m}')

# Train Faces Sample Size: 485
# Train Back Sample Size: 909
# m: 884


In [10]:
sample_train_faces = random.sample(train_faces,n_face)

Im_train = []
for filename in tqdm(sample_train_faces):
    path = '/content/train/face/' + filename
    with open(path, 'rb') as pgmf:
        image = plt.imread(pgmf)
    Im_train.append(image)

n_train_faces = len(Im_train)
y_train = [1]*n_train_faces

100%|██████████| 485/485 [00:00<00:00, 3876.11it/s]


In [13]:
sample_train_background = random.sample(train_background,n_back)

for filename in tqdm(sample_train_background):
    path = "/content/train/non-face/" + filename
    with open(path, 'rb') as pgmf:
        image = plt.imread(pgmf)
    Im_train.append(image)

n_train_background = len(Im_train)-n_train_faces
y_train = y_train + [0]*n_train_background

100%|██████████| 909/909 [00:00<00:00, 5934.85it/s]


In [15]:
print(f'# Train: {len(Im_train)}, {len(y_train)}')

# Train: 1394, 1394


In [11]:
Im_test = []
for filename in tqdm(test_faces):
    path = "/content/test/face/" + filename
    with open(path, 'rb') as pgmf:
        image = plt.imread(pgmf)
    Im_test.append(image)

n_test_faces = len(Im_test)
y_test = [1]*n_test_faces

100%|██████████| 472/472 [00:00<00:00, 3754.54it/s]


In [12]:
sample_test_background = random.sample(test_background,m)

for filename in tqdm(sample_test_background):
    path = "/content/test/non-face/" + filename
    with open(path, 'rb') as pgmf:
        image = plt.imread(pgmf)
    Im_test.append(image)

n_test_background = len(Im_test)-n_test_faces
y_test = y_test + [0]*n_test_background

100%|██████████| 884/884 [00:00<00:00, 3462.70it/s]


In [16]:
print(f'# Test: {len(Im_test)}, {len(y_test)}')

# Test: 1356, 1356


## Histogram equalization

In [18]:
Im_train_norm = [equalize_hist(image) for image in Im_train]
Im_test_norm = [equalize_hist(image) for image in Im_test]

## Matriz de features

### Calculamos y seleccionamos las mejores features en entrenamiento

In [21]:
X_train = [extract_feature_image(img) for img in tqdm(Im_train_norm)]
X_train = np.array(X_train)

100%|██████████| 1394/1394 [04:10<00:00,  5.56it/s]


In [22]:
# Pueden guardar la matriz si lo desean
np.save('X_train', X_train)

In [24]:
# Y cargarla posteriormente
X_train = np.load('X_train.npy')

In [25]:
X_train.shape

(1394, 63960)

In [31]:
print("Seleccionando las features de mayor dependencia lineal con y")
t_start = time()
f_indices = SelectPercentile(f_classif, percentile=1).fit(X_train, y_train).get_support(indices=True)
t = time() - t_start
X_train = X_train[:,f_indices]
print("Seleccionadas %d features potenciales" % X_train.shape[1])
print(f'Tiempo: {t} segundos')

Seleccionando las features de mayor dependencia lineal con y
Seleccionadas 640 features potenciales
Tiempo: 1.376927137374878 segundos


### Calculamos dichas features para test

In [35]:
feature_coord, feature_type = haar_like_feature_coord(width=19,
                                                      height=19,
                                                      )

In [36]:
t_start = time()
X_test = [extract_feature_image(img,
                                feature_type=feature_type[f_indices],
                                feature_coord=feature_coord[f_indices]) for img in tqdm(Im_test_norm)]
t = time() - t_start
X_test = np.array(X_test)

100%|██████████| 1356/1356 [00:01<00:00, 704.43it/s]


In [37]:
print(f'Tiempo: {t} segundos')
print(f'Shape X_test: {X_test.shape}')

Tiempo: 1.936858892440796 segundos
Shape X_test: (1356, 640)
